In [1]:
import pandas as pd

df = pd.read_csv('https://stepik.org/media/attachments/lesson/1110806/vacancies_messages_50.csv')

In [2]:
!pip install langchain langchain-openai openai tiktoken -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter 1.0.0 requires qtconsole, which is not installed.


In [3]:
from tqdm import tqdm
import os
from getpass import getpass
import warnings
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain import PromptTemplate


warnings.filterwarnings('ignore')

In [4]:
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

--2024-05-04 19:36:56--  https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 11184 (11K) [text/plain]
Сохранение в: «utils.py»

utils.py            100%[===================>]  10,92K  --.-KB/s    за 0,001s  

2024-05-04 19:36:57 (13,0 MB/s) - «utils.py» сохранён [11184/11184]



In [5]:
# Если используете ключ из курса, запустите эту ячейку
from utils import ChatOpenAI

#course_api_key= "Введите ваш OpenAI API ключ"
course_api_key = getpass(prompt='Введите API ключ')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите API ключ········


In [23]:
input_text = """
#вакансия #vacancy #Python #удаленка #flask #relocation #senior
Позиция: Senior Python developer
Компания: Collectly Формат работы: Удаленно (не на территории РФ/РБ)
Вилка: 6000-9000$.net Трудоустройство: ИП или ТК Армении Занятость: fulltime
Контакты: @ann_gfio Collectly - американская технологическая компания, прошедшая акселератор Y Combinator, специализирующаяся на автоматизации биллинга в медицинской сфере США. За почти 4 года существования компания стала прибыльной и стабильной, а также приобрела статус быстрорастущей организации.
Collectly - первая в мире система, обеспечивающая автономное взаимодействие американских медицинских учреждений с пациентами в отношении медицинских счетов. Компания помогает пациентам разобраться в страховых планах, оплатить счета и получить финансовую помощь, снижая операционные расходы медицинских учреждений.
Работа компании Collectly решает важные социальные проблемы в США и получает признание ведущих отраслевых журналов.
🔎 Кого ищем? Мы ищем сильного инженера, который готов работать в маленькой, но быстрорастущей технической команде: нужно уметь решать практические задачи просто (=> быстро), при этом свобода выбора за решение лежит на вас.
Но и ответственность тоже! Процессы и инструменты: - Недельные спринты с одним большим совещанием. JIRA для задач, Slack и Zoom для общения; - Внедрен CI/CD на основе Gitlab, тесты в пайплайне.
Релизы каждой фичи независимые. Кто фичу писал — тот за неё и ответственный; - Code review перед Merge Request, посмотреть может каждый.
Стек: Python, Flask, SqlAlchemy, PostgreSQL, Marshmallow, Celery, RabbitMQ, Redis. Custom REST API framework.
Требования: - У вас должен быть практический опыт коммерческой web разработки от 4 лет и более; - Есть большой опыт работы с SQLAlchemy; - Хорошо знаете английский язык - В2 и выше; - Опыт работы с асинхронным кодом.
Мы предлагаем: - Полностью удаленная full-time вакансия; - Опцион (право на получение акций) в американском стартапе
"""



template = """\
Тебе передан текст объявления о вакансии на работу. Из текста извлеки информацию:

job_title: Наименование вакансии (позиции) указанной в тексте. Наименование позиции должно быть указано без грейда (Senior, middle, Lead и прочее). Должно содержать только наименование вакансии. Если не найдешь укажи None

company: Наименование компании. Необхощимо указать только название (не пишем финтех, крупная компания, мобильная игра и т.д.). Если не найдешь укажи None

salary: Указать заработную плату. Писать только числа и валюту. Числа писать без пробелов. После числа (диапазона чисел) указываем валюту руб. или $ после пробела. Если указана только нижняя граница: от, выше, то необходимо написать  слово  от. Если указана только верхняя граница, то пишем это значение, используя слово до. Не писать: фикс, плюшки, премии, % от продаж, техника и т.д.  Если указано тыс. или к. необходимо умножить на 1000. Не писать net, gross, на руки, тыс., к. и т.д. Если указан диапазон, то писать через тире, около тире пробелы не ставить. Если зарплата указана за час, то в конце добавляем в час.  Если не найдешь укажи None

tg: Указать контакты для связи. Если указано несколько контаков, то перечислить их через запятую. Если не найдешь укажи None

grade: Указать требуемый грейд. Возможные значения intern, junior, junior+, middle, middle+, senior, lead.
Если указано несколько значений, то пишем их через запятую в порядке возрастания. Если не найдешь укажи None

text: {text}

{format_instructions}

"""

In [7]:
job_title_schema = ResponseSchema(name="job_title",
                             description="Наименование вакансии (позиции) указанной в тексте. Наименование позиции должно быть указано без грейда (Senior, middle и прочее). Должно содержать только наименование вакансии.")

company_days_schema = ResponseSchema(name="company",
                                      description="Наименование компании. Необхощимо указать только название (не пишем финтех, крупная компания, мобильная игра и т.д.)")

salary_schema = ResponseSchema(name="salary",
                                    description="Указать заработную плату.")

tg_schema = ResponseSchema(name="tg",
                                    description="Указать контакты для связи. Если указано несколько контаков, то перечислить их через запятую")

grade_schema = ResponseSchema(name="grade",
                                    description="Указать требуемый грейд. Возможные значения intern, junior, junior+, middle, middle+, senior, lead.Если указано несколько значений, то пишем их через запятую в порядке возрастания.")

response_schemas = [job_title_schema,
                    company_days_schema,
                    salary_schema,
                    tg_schema,
                    grade_schema]

In [24]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
prompt_template = ChatPromptTemplate.from_template(template)

In [25]:
messages = prompt_template.format_messages(text=input_text,
                                format_instructions=format_instructions)

In [10]:
chat = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

In [11]:
response = chat.invoke(messages)
print(response.content)

```json
{
	"job_title": "Python developer",
	"company": "Collectly",
	"salary": "6000-9000$",
	"tg": "@ann_gfio",
	"grade": "senior"
}
```


In [12]:
output_dict = output_parser.parse(response.content)
print(output_dict)
output_dict.get('job_title')


{'job_title': 'Python developer', 'company': 'Collectly', 'salary': '6000-9000$', 'tg': '@ann_gfio', 'grade': 'senior'}


'Python developer'

Цикл

In [29]:
job_title_list = [] # Список, где будем хранить ответы модели
company_list = []
salary_list = []
tg_list = []
grade_list = []

i = 0
for context in tqdm(df['text']):
    messages = prompt_template.format_messages(text=context,
                                format_instructions=format_instructions) # Добавляем сообщение в промпт
    response = chat.invoke(messages) # Ответ модели
    output_dict = output_parser.parse(response.content)

    job_title_list.append(output_dict.get('job_title')) # Добавляем ответ в список
    company_list.append(output_dict.get('company'))
    salary_list.append(output_dict.get('salary'))
    tg_list.append(output_dict.get('tg'))
    grade_list.append(output_dict.get('grade'))
    i=i+1
    #if i == 5:
    #  break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:27<00:00,  1.75s/it]


In [30]:
print(job_title_list)
print(company_list)
print(salary_list)
print(tg_list)
print(grade_list)

['Python developer', 'Системный аналитик', 'Database Administrator', 'Руководителя отдела системного администрирования', 'Analyst', 'DevOps инженер', 'Системный аналитик', 'Python/Django full-stack разработчика', 'Go разработчик', 'Разработчик', 'Lead UI/UX Designer', 'QA automation Engineer', 'Full Stack PHP-разработчик', 'Frontend разработчик', 'React Developer', 'PHP-developer', 'Retention Manager', 'Python developer', 'Программист 1С:ЗУП', 'Системный аналитик', 'Tech Lead QA', 'Backend Python разработчик', 'Менеджер по развитию партнерской сети', 'Full-stack разработчик', 'C# Engineer', 'System Analyst', 'Разработчик PowerBuilder', 'Системный аналитик', 'Java developer', 'PHP программист', 'IT Recruiter', 'C/C++ Developer', 'PHP developer', 'Python Developer', 'Go Developer', 'SDR', 'C# Developer', 'Системный администратор', 'Аналитик', 'Копирайтер в сфере крипты', 'QA', 'Системный аналитик', 'Release manager', 'DevOps (Openstack Engineer)', 'Go Developer', 'Backend Engineer', 'про

In [31]:
df['job_title'] = job_title_list
df['company'] = company_list
df['salary'] = salary_list
df['tg'] = tg_list
df['grade'] = grade_list

In [32]:
df.to_csv('/Users/alexeyvaganov/Downloads/2_2_9_solution.csv', index=False)

Конец цикла

In [28]:
context = df.iloc[10]["text"]
messages = prompt_template.format_messages(text=context,
                                format_instructions=format_instructions)
response = chat.invoke(messages) # Ответ модели
output_dict = output_parser.parse(response.content)
print(context)
print(output_dict)

#vacancy #relocate #fulltime #LeadUIUXDesigner #gamedev
Вакансия для Lead UI/UX Designer.
C чем работать: мобильная игра в стиле merge,казуалка.
Оформление: штат.
🖌Что будет важным:
 - Опыт разработки коммерческих интерфейсов более 3-х лет в gamedev.
 - Умение проектировать интерфейсы мобильных игр с нуля; 
 - Опыт работы с Figma, Photoshop, Unity будет плюсом; 
 - Навыки 2D и иллюстрации; 
 - Умение создавать лаконичную и красивую иконографию.
Буду рада ответить на все вопросы 📩 @alesyaly
{'job_title': 'Lead UI/UX Designer', 'company': 'None', 'salary': 'None', 'tg': '@alesyaly', 'grade': 'lead'}


In [18]:
df

,text_id,text
0,9,#вакансия #vacancy #Python #удаленка #flask #r...
1,31,#ВАКАНСИЯ #Системный_аналитик #РФ\n \nАКЦИОНЕ...
2,28,#вакансия #vacancy #job #senior #data #DB #dat...
3,49,#вакансия #fulltime #remote \n\n🔎 Ищем Руковод...
4,18,#vacancy #job #analyst #travel #sirenatravel #...
5,48,#вакансия #fulltime #remote #lookfor #devops\n...
6,41,#вакансия #senior #middle+ #lead #удаленка #оф...
7,6,#job #python #django #javascript #react #fulls...
8,17,#вакансия #vacancy #middle #senior #remote #уд...
9,4,Вакансия: Разработчик (Vue.js / Golang) для на...
